In [1]:
!pip install schedule
!pip install azure-eventhub==5.11.5

import numpy as np
import pandas as pd
import requests
import re
import torch
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaModel, RobertaTokenizer
import openai
import schedule
import time
from azure.eventhub import EventHubProducerClient, EventData
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from openai.error import RateLimitError
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from datetime import datetime
from dateutil.parser import parse, isoparse

spark = SparkSession.builder \
    .appName('App') \
    .getOrCreate()


StatementMeta(, 90e6cb92-be31-4e38-b618-5de7b1032a90, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.


2024-03-20 07:13:03.613341: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-20 07:13:08.993584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:

# Set up Azure OpenAI API credentials
openai.api_version = "2024-02-15-preview"
openai.api_base = "https://dqdemo01.openai.azure.com/"
openai.api_type = "azure"
openai.api_key = "14df843620644b0ab178768a855e0b79"
deployment_id = "Analytics"

# Set up FinBERT
tokenizer_finbert = AutoTokenizer.from_pretrained('ProsusAI/finbert')
finbert_model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')

# Set up RoBERTa
tokenizer_roberta = AutoTokenizer.from_pretrained('roberta-base')
roberta_model = AutoModelForSequenceClassification.from_pretrained('roberta-base')

def send_to_eventhub(csv_data):
    # Your Azure Event Hub connection string and event hub name
    connection_str = "Endpoint=sb://ehns-727fd2-fabricworkshop.servicebus.windows.net/;SharedAccessKeyName=StockMarketData;SharedAccessKey=d7eUe1evQ3nCrTT5S5i7vYmDZf4xEzjFg+AEhMu6wpk=;EntityPath=stockeventhub"
    eventhub_name = "stockeventhub"

    producer = EventHubProducerClient.from_connection_string(connection_str, eventhub_name=eventhub_name)

    with producer:
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(csv_data))
        producer.send_batch(event_data_batch)
    print("Data sent to Azure Event Hub successfully.")

def finbert_model_scores(article_text):
    inputs = tokenizer_finbert(article_text, padding=True, truncation=True, return_tensors='pt')
    outputs = finbert_model(**inputs)

    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    positive = predictions[:, 0].item()
    negative = predictions[:, 1].item()
    # Calculate the sentiment score
    sentiment_score = positive - negative

    return sentiment_score

def roberta_model_scores(article_text):
    # Tokenize the input text
    inputs = tokenizer_roberta(article_text, return_tensors="pt", max_length=512, truncation=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Perform sentiment analysis
    inputs = {key: inputs[key].to(device) for key in inputs}
    outputs = roberta_model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    positive_prob = probabilities[0, 1].item()  # Access the probability for positive sentiment
    negative_prob = probabilities[0, 0].item()  # Access the probability for negative sentiment
    sentiment_score = positive_prob - negative_prob

    return sentiment_score

def get_openai_response(article_text, stock_symbol):
    chatgpt_input = f"Please provide a sentiment score for the following news article related to stock {stock_symbol}: /n/n {article_text}/n/n Provide a single float value as the sentiment score based on the following guideline: If there is no discernible effect of the news on the stock, return 0. If the news has a negative effect on the stock, return a float value between -1 and 0. If the news has a positive effect on the stock, return a float value between 0 and 1."
    try:
        response = openai.ChatCompletion.create(
            deployment_id=deployment_id,
            messages=[
                {"role": "user", "content": chatgpt_input}
            ]
        )
        assistant_response = response['choices'][0]['message']['content']
        try:
            # Try to convert the response to a float
            sentiment_score = float(assistant_response)
        except ValueError:
            # If conversion fails (e.g., response is not a valid number), set sentiment_score to 0
            sentiment_score = 0
    except RateLimitError as e:
        print("Rate limit exceeded. Waiting for 6 seconds before retrying.")
        time.sleep(6)
        sentiment_score = get_openai_response(article_text, stock_symbol)
    return sentiment_score

# Function to fetch news articles and perform sentiment analysis
def fetch_and_print_sentiment_scores():
    kustoQuery = "['StockPrice'] | take 5000"
    kustoUri = "https://trd-uefxabgf86b291we8f.z5.kusto.fabric.microsoft.com"
    database = "RealTimeStockData"
    accessToken = mssparkutils.credentials.getToken(kustoUri)
    kustoDf  = spark.read\
        .format("com.microsoft.kusto.spark.synapse.datasource")\
        .option("accessToken", accessToken)\
        .option("kustoCluster", kustoUri)\
        .option("kustoDatabase", database)\
        .option("kustoQuery", kustoQuery).load()

    if not isinstance(kustoDf, pyspark.sql.dataframe.DataFrame):
        print("Error: kustoDf is not a DataFrame.")
        return

    #if 'URL' not in kustoDf.columns:
    #    print("Error: 'URL' column not found in kustoDf.")
    #    return
    # List of stock symbols
    stock_symbols = ['CSCO', 'ADSK', 'INTC', 'ADBE', 'WMT', 'NFLX', 'DIS' , 'KO', 'AMZN', 'META']
    # News API URL and API token
    api_token = "10iKgRj77CdcYyRN9XGMxJ2orvtFBHgad8RQsSFb"
    url = f"https://api.marketaux.com/v1/news/all?filter_entities=true&language=en&api_token={api_token}&country=us"

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error in response from news API: Status Code {response.status_code}")
        return
    data = response.json()
    articles = data.get('data', [])
    if not articles:
        print("No articles found in the API response.")
        return

    for article in articles:
        article_url = article.get('url', '')
        published_at = article.get('published_at', '')

        if published_at.endswith('Z'):
            published_at = published_at[:-1]
        try:

            published_datetime = isoparse(published_at)
        except ValueError as e:
            print(f"Error parsing published datetime for article: {e}")
            continue

        formatted_published_at = published_datetime.strftime("%Y-%m-%d %H:%M:%S")

        # Check if the URL has already been processed
        #if article_url in kustoDf.select('URL').rdd.flatMap(lambda x: x).collect():
         #   print('Article already processed:', article_url)
          #  continue  # Skip processing this article and move to the next one
         # Process the new URL
        response_article = requests.get(article_url)
        soup_article = BeautifulSoup(response_article.content, 'html.parser')
        article_text = ' '.join([p.get_text() for p in soup_article.find_all('p')])
        data_list = []  # List to store data for each article
        # Perform sentiment analysis
        for stock_symbol in stock_symbols:
            finbert_score = finbert_model_scores(article_text)
            roberta_score = roberta_model_scores(article_text)
            chatgpt_score = get_openai_response(article_text, stock_symbol)

            data_list.append({
                'Headline': article['title'],
                'Description': article['description'],
                'Source': article['source'],
                'Published At': published_datetime.strftime("%Y-%m-%d %H:%M:%S"),
                'URL': article_url,
                'Stock Symbol': stock_symbol,
                'FinBERT Score': finbert_score,
                'RoBERTa Score': roberta_score,
                'ChatGPT Score': float(chatgpt_score),  # Explicitly cast to DoubleType
                'Combined Score': 0.60 * float(chatgpt_score) + 0.20 * float(finbert_score) + 0.20 * float(roberta_score)
            })

        # Create DataFrame
        schema = StructType([
            StructField('Headline', StringType(), True),
            StructField('Description', StringType(), True),
            StructField('Source', StringType(), True),
            StructField('Published At', StringType(), True),
            StructField('URL', StringType(), True),
            StructField('Stock Symbol', StringType(), True),
            StructField('FinBERT Score', DoubleType(), True),
            StructField('RoBERTa Score', DoubleType(), True),
            StructField('ChatGPT Score', DoubleType(), True),
            StructField('Combined Score', DoubleType(), True)
        ])
        df = spark.createDataFrame(data_list, schema=schema)

        # Convert DataFrame to JSON
        df_pandas = df.toPandas()

        # Convert Pandas DataFrame to CSV format
        csv_data = df_pandas.to_csv(index=False)

        # Send CSV data to Azure Event Hub
        #send_to_eventhub(csv_data)

        # Create DataFrame with the same schema as kustoDf
        #schema = kustoDf.schema
        #new_urls_df = spark.createDataFrame([], schema=schema)

        # Add processed URL to new_urls_df
        for _ in range(len(stock_symbols)):
            new_row = (None,) * len(schema.fields)  # Create a row with None values for each column
            #new_urls_df = new_urls_df.union(spark.createDataFrame([new_row], schema=schema))
            print(new_row)

        # Union operation
        #kustoDf = kustoDf.union(new_urls_df)

    # Display the DataFrame
    display(kustoDf)
    df.show()
    return df

fetch_and_print_sentiment_scores()

StatementMeta(, 90e6cb92-be31-4e38-b618-5de7b1032a90, 20, Finished, Available)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None, None, None)
(None, None, None, None,

SynapseWidget(Synapse.DataFrame, 45c17184-d6a6-4e6e-bf33-c4ba3b0f642c)

+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------------+-------------------+-------------+------------------+
|            Headline|         Description|              Source|       Published At|                 URL|Stock Symbol|     FinBERT Score|      RoBERTa Score|ChatGPT Score|    Combined Score|
+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------------+-------------------+-------------+------------------+
|More than half of...|Artificial intell...|retailtechnologyr...|2024-03-20 07:33:46|https://www.retai...|        CSCO|0.4717923775315285|0.05991411209106445|          0.8|0.5863412979245186|
|More than half of...|Artificial intell...|retailtechnologyr...|2024-03-20 07:33:46|https://www.retai...|        ADSK|0.4717923775315285|0.05991411209106445|          0.7|0.5263412979245186|
|More than half of...|Artificial intell...|re

DataFrame[Headline: string, Description: string, Source: string, Published At: string, URL: string, Stock Symbol: string, FinBERT Score: double, RoBERTa Score: double, ChatGPT Score: double, Combined Score: double]